In [1]:
from transformers import pipeline

c:\Users\ganga\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
model1_name = "cardiffnlp/twitter-roberta-base-sentiment"
sa_pipeline1 = pipeline("sentiment-analysis", model=model1_name)

c:\Users\ganga\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ganga\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
model2_name = "distilbert-base-uncased-finetuned-sst-2-english"
sa_pipeline2 = pipeline("sentiment-analysis", model=model2_name)

In [4]:
label_map = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "NEUTRAL",
    "LABEL_2": "POSITIVE"
}

In [5]:
texts = [
    "I absolutely love this product! It's amazing.",
    "The experience was terrible, I would not recommend it.",
    "The movie was okay, not the best but not the worst either.",
    "This service is exceptional and exceeded my expectations.",
    "I'm very disappointed with the quality of the item."
]

In [6]:
def ensemble_sentiment(text):
    result1 = sa_pipeline1(text)[0]
    result2 = sa_pipeline2(text)[0]

    label1 = label_map.get(result1["label"], result1["label"])
    label2 = result2["label"].capitalize()

    if label1 == label2:
        final_label = label1
    else:
        final_label = label1 if result1["score"] >= result2["score"] else label2

    return {
        "text": text,
        "model1": {"label": label1, "score": result1["score"]},
        "model2": result2,
        "ensemble": final_label
    }

In [7]:
for text in texts:
    result = ensemble_sentiment(text)
    print(f"Text: {result['text']}")
    print(f" Model 1 ({model1_name}): {result['model1']}")
    print(f" Model 2 ({model2_name}): {result['model2']}")
    print(f" Ensemble Sentiment: {result['ensemble']}")
    print("-" * 60)

Text: I absolutely love this product! It's amazing.
 Model 1 (cardiffnlp/twitter-roberta-base-sentiment): {'label': 'POSITIVE', 'score': 0.9922341108322144}
 Model 2 (distilbert-base-uncased-finetuned-sst-2-english): {'label': 'POSITIVE', 'score': 0.9998855590820312}
 Ensemble Sentiment: Positive
------------------------------------------------------------
Text: The experience was terrible, I would not recommend it.
 Model 1 (cardiffnlp/twitter-roberta-base-sentiment): {'label': 'NEGATIVE', 'score': 0.9805479049682617}
 Model 2 (distilbert-base-uncased-finetuned-sst-2-english): {'label': 'NEGATIVE', 'score': 0.9979416728019714}
 Ensemble Sentiment: Negative
------------------------------------------------------------
Text: The movie was okay, not the best but not the worst either.
 Model 1 (cardiffnlp/twitter-roberta-base-sentiment): {'label': 'NEUTRAL', 'score': 0.4815261960029602}
 Model 2 (distilbert-base-uncased-finetuned-sst-2-english): {'label': 'NEGATIVE', 'score': 0.84922790527